# Visualizing Debugging Tensors of MXNet training

### Overview

SageMaker Debugger is a new capability of Amazon SageMaker that allows debugging machine learning models. 
It lets you go beyond just looking at scalars like losses and accuracies during training and gives 
you full visibility into all the tensors 'flowing through the graph' during training. SageMaker Debugger helps you to monitor your training in near real time using rules and would provide you alerts, once it has detected an inconsistency in the training flow.

Using SageMaker Debugger is a two step process: Saving tensors and Analysis. In this notebook we will run an MXNet training job and configure SageMaker Debugger to store all tensors from this job. Afterwards we will visualize those tensors in our notebook.


### Dependencies
Before we begin, let us install the library plotly if it is not already present in the environment.
If the below cell installs the library for the first time, you'll have to restart the kernel and come back to the notebook. In addition to that, in order for our vizualiation to access tensors let's install smdebug - debugger library that provides API access to tensors emitted during training job.

In [1]:
! pip install --upgrade nbformat
! python -m pip install plotly
! python -m pip install smdebug


Requirement already up-to-date: nbformat in /usr/local/lib/python3.6/site-packages (5.1.3)
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Configure and run the training job

Now we'll call the Sagemaker MXNet Estimator to kick off a training job with Debugger attached to it.

The `entry_point_script` points to the MXNet training script.

In [2]:
import os
import sagemaker
from sagemaker.debugger import DebuggerHookConfig, CollectionConfig
from sagemaker.mxnet import MXNet

sagemaker_session = sagemaker.Session()

entry_point_script = "./scripts/mxnet_gluon_save_all_demo.py"
hyperparameters = {"batch-size": 256}
base_job_name = "mnist-tensor-plot"

# Make sure to set this to your bucket and location
BUCKET_NAME = 'dl-workshop-aug2021'
LOCATION_IN_BUCKET = "debugger"
s3_bucket_for_tensors = "s3://{BUCKET_NAME}/{LOCATION_IN_BUCKET}".format(
    BUCKET_NAME=BUCKET_NAME, LOCATION_IN_BUCKET=LOCATION_IN_BUCKET
)

estimator = MXNet(
    role=sagemaker.get_execution_role(),
    base_job_name=base_job_name,
    train_instance_count=1,
    train_instance_type="ml.m4.xlarge",
    entry_point=entry_point_script,
    framework_version="1.6.0",
    train_max_run=3600,
    sagemaker_session=sagemaker_session,
    py_version="py3",
    debugger_hook_config=DebuggerHookConfig(
        s3_output_path=s3_bucket_for_tensors,  # Required
        collection_configs=[
            CollectionConfig(
                name="all_tensors", parameters={"include_regex": ".*", "save_steps": "1, 2, 3"}
            )
        ],
    ),
)

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Estimator described above will save all tensors of all layers during steps 1, 2 and 3. Now, let's start the training job:

In [3]:
estimator.fit()

2021-08-18 16:17:37 Starting - Starting the training job...
2021-08-18 16:18:00 Starting - Launching requested ML instancesProfilerReport-1629303456: InProgress
...
2021-08-18 16:18:34 Starting - Preparing the instances for training.........
2021-08-18 16:20:06 Downloading - Downloading input data...
2021-08-18 16:20:22 Training - Downloading the training image..2021-08-18 16:20:46,614 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
2021-08-18 16:20:46,616 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-08-18 16:20:46,631 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{}', 'SM_USER_ENTRY_POINT': 'mxnet_gluon_save_all_demo.py', 'SM_FRAMEWORK_PARAMS': '{}', 'SM_RESOURCE_CONFIG': '{"current_host":"algo-1","hosts":["algo-1"],"network_interface_name":"eth0"}', 'SM_INPUT_DATA_CONFIG': '{}', 'SM_OUTPUT_DATA_DIR': 

### Get S3 location of tensors

Now we can retrieve the S3 location of the tensors:

In [4]:
tensors_path = estimator.latest_job_debugger_artifacts_path()
print("S3 location of tensors is: ", tensors_path)

S3 location of tensors is:  s3://dl-workshop-aug2021/debugger/mnist-tensor-plot-2021-08-18-16-17-36-655/debug-output


### Download tensors from S3

Next we download the tensors from S3, so that we can visualize them in the notebook.

In [5]:
folder_name = tensors_path.split("/")[-1]
os.system("aws s3 cp --recursive " + tensors_path + " " + folder_name)
print("Downloaded tensors into folder: ", folder_name)

Downloaded tensors into folder:  debug-output


### Visualize
The main purpose of this class (TensorPlot) is to visualise the tensors in your network. This could be to determine dead or saturated activations, or the features maps the network.

To use this class (TensorPlot), you will need to supply the argument regex with the tensors you are interested in. e.g., if you are interested in activation outputs, then you need to supply the following regex .*relu|.*tanh|.*sigmoid.

Another important argument is the `sample_batch_id`, which allows you to specify the index of the batch size to display. For example, given an input tensor of size (batch_size, channel, width, height), `sample_batch_id = n` will display (n, channel, width, height). If you set sample_batch_id = -1 then the tensors will be summed over the batch dimension (i.e., `np.sum(tensor, axis=0)`). If batch_sample_id is None then each sample will be plotted as separate layer in the figure.

Here are some interesting use cases:

1) If you want to determine dead or saturated activations for instance ReLus that are always outputting zero, then you would want to sum the batch dimension (sample_batch_id=-1). The sum gives an indication which parts of the network are inactive across a batch.

2) If you are interested in the feature maps for the first image in the batch, then you should provide batch_sample_id=0. This can be helpful if your model is not performing well for certain set of samples and you want to understand which activations are leading to misprediction.

An example visualization of layer outputs:
![](./images/tensorplot.gif)


`TensorPlot` normalizes tensor values to the range 0 to 1 which means colorscales are the same across layers. Blue indicates value close to 0 and yellow indicates values close to 1. This class has been designed to plot convolutional networks that take 2D images as input and predict classes or produce output images. You can use this  for other types of networks like RNNs, but you may have to adjust the class as it is currently neglecting tensors that have more than 4 dimensions.

Let's plot Relu output activations for the given MNIST training example.

In [6]:
import tensor_plot

visualization = tensor_plot.TensorPlot(
    regex=".*relu_output",
    path=folder_name,
    steps=10,
    batch_sample_id=0,
    color_channel=1,
    title="Relu outputs",
    label=".*sequential0_input_0",
    prediction=".*sequential0_output_0",
)

[2021-08-18 16:21:52.679 mxnet-1-6-cpu-py36-ml-t3-medium-e13e5c32ffa52b276f0ed89d7b7a:161 INFO local_trial.py:35] Loading trial debug-output at path debug-output
[2021-08-18 16:21:52.737 mxnet-1-6-cpu-py36-ml-t3-medium-e13e5c32ffa52b276f0ed89d7b7a:161 INFO trial.py:198] Training has ended, will refresh one final time in 1 sec.
[2021-08-18 16:21:53.743 mxnet-1-6-cpu-py36-ml-t3-medium-e13e5c32ffa52b276f0ed89d7b7a:161 INFO trial.py:210] Loaded all steps


If we plot too many layers, it can crash the notebook. If you encounter performance or out of memory issues, then either try to reduce the layers to plot by changing the `regex` or run this Notebook in JupyterLab instead of Jupyter. 

In the below cell we vizualize outputs of all layers, including final classification. Please note that because training job ran only for 1 epoch classification accuracy is not high.

In [7]:
visualization.fig.show(renderer="iframe")

In [8]:
!pip freeze

attrs==20.3.0
aws-mxnet-mkl==1.6.0
awscli==1.19.20
backcall==0.2.0
bcrypt==3.2.0
boto3==1.17.20
botocore==1.20.20
certifi==2020.12.5
cffi==1.14.5
chardet==3.0.4
cloudpickle==1.6.0
colorama==0.4.3
cryptography==3.4.6
cycler==0.10.0
Cython==0.29.22
decorator==4.4.2
dgl==0.4.3.post2
docutils==0.15.2
gevent==21.1.2
gluoncv==0.6.0
gluonnlp==0.10.0
google-pasta==0.2.0
graphviz==0.8.4
greenlet==1.0.0
h5py==2.10.0
horovod==0.19.5
idna==2.8
importlib-metadata==3.7.0
inotify-simple==1.2.1
ipykernel==5.5.5
ipython==7.16.1
ipython-genutils==0.2.0
jedi==0.18.0
jmespath==0.10.0
jsonschema==3.2.0
jupyter-client==6.2.0
jupyter-core==4.7.1
Keras-Applications==1.0.8
keras-mxnet==2.2.4.2
Keras-Preprocessing==1.1.2
kiwisolver==1.3.1
matplotlib==3.3.4
mpi4py==3.0.2
nbformat==5.1.3
nest-asyncio==1.5.1
networkx==2.5
numpy==1.19.1
onnx==1.6.0
packaging==20.9
pandas==0.25.1
paramiko==2.7.2
parso==0.8.2
pexpect==4.8.0
pickleshare==0.7.5
Pillow==8.1.1
plotly==5.2.1
portalocker==2.2.1
prompt-toolkit==3.0.19
proto

For additional example of working with debugging tensors and visualizing them in real time please feel free to try it out at [MXNet realtime analysis](../mxnet_realtime_analysis/mxnet-realtime-analysis.ipynb) example.